### Source of data
##### Golub et al. (1999). Molecular classification of cancer: class discovery and class prediction by gene expression monitoring, Science, Vol. 286:531-537.

In [2]:
import matplotlib.pyplot
import pandas
import numpy

In [9]:
data = pandas.read_csv('./data/data.csv').drop('Unnamed: 0', axis = 1)
data_class = pandas.read_csv('./data/data_class.csv').drop('Unnamed: 0', axis = 1)

In [38]:
ALL_data = data.loc[:,list(data_class['x'] == 0)]
AML_data = data.loc[:,list(data_class['x'] == 1)]

In [52]:
ALL_mean = numpy.mean(ALL_data, axis = 1)
AML_mean = numpy.mean(AML_data, axis = 1)

ALL_var = numpy.var(ALL_data, axis = 1, ddof = 1)
AML_var = numpy.var(AML_data, axis = 1, ddof = 1)

ALL_count = ALL_data.shape[1]
AML_count = AML_data.shape[1]

ALL_S = ALL_var / ALL_count
AML_S = AML_var / AML_count

In [98]:
from scipy.stats import t

delta_mean = numpy.abs(ALL_mean - AML_mean)
delta_S = ALL_S + AML_S

t_welch = delta_mean / (ALL_var / ALL_count + AML_var / AML_count) ** 0.5
t_welch_ddof = ((ALL_var / ALL_count + AML_var / AML_count) ** 2) / ((ALL_var / ALL_count) ** 2 / (ALL_count - 1) + (AML_var / ALL_count) ** 2 / (AML_count - 1))
t_welch_ddof = numpy.round(t_welch_ddof)

significance = 0.05
reject_region = t.ppf(1 - significance/2, t_welch_ddof)
p_value = t.sf(t_welch, t_welch_ddof)

print(f'Number of significant genes (p-value): {len(p_value[p_value <= significance / 2])}')
print(f'Number of significant genes (rejection): {len(t_welch[t_welch >= reject_region])}')

Number of significant genes (p-value): 1142
Number of significant genes (rejection): 1142


In [100]:
from scipy.stats import ttest_ind

t_welch, p_value = ttest_ind(ALL_data, AML_data, axis=1, equal_var=False)
significant_genes = numpy.concatenate((p_value[p_value <= significance], p_value[p_value >= 1 - significance]))
print(f'Number of significant genes (scipy): {len(significant_genes)}')

Number of significant genes (scipy): 1149


In [102]:
from statsmodels.stats.multitest import multipletests

HM_correction = multipletests(p_value, alpha=significance, method='holm')
significant_HM_p_values = t_welch[HM_correction[0]]
print(f'Number of significant genes (Holm-Bonferroni correction): {len(significant_HM_p_values)}')

BH_correction = multipletests(p_value, alpha=significance, method='fdr_bh')
significant_BH_p_values = t_welch[BH_correction[0]]
print(f'Number of significant genes (Benjamini-Hochberg correction): {len(significant_BH_p_values)}')

Number of significant genes (Holm-Bonferroni correction): 103
Number of significant genes (Benjamini-Hochberg correction): 695
